# Day 09. Exercise 02
# Metrics

## 0. Imports

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from collections import Counter

## 1. Preprocessing

1. Create the same dataframe as in the previous exercise.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [9]:
data = pd.read_csv('../data/day-of-week-not-scaled.csv')
df = pd.read_csv("../data/dayofweek.csv")

X = data
y = df['dayofweek']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=21, stratify=y)

## 2. SVM

1. Use the best parameters from the previous exercise and train the model of SVM.
2. You need to calculate `accuracy`, `precision`, `recall`, `ROC AUC`.

 - `precision` and `recall` should be calculated for each class (use `average='weighted'`)
 - `ROC AUC` should be calculated for each class against any other class (all possible pairwise combinations) and then weighted average should be applied for the final metric
 - the code in the cell should display the result as below:

```
accuracy is 0.88757
precision is 0.89267
recall is 0.88757
roc_auc is 0.97878
```

In [13]:
best_svm_params = {
    'C': 10, 
    'class_weight': None, 
    'gamma': 'auto', 
    'kernel': 'rbf'
}

svm_model = SVC(C=best_svm_params["C"], kernel=best_svm_params["kernel"], gamma=best_svm_params["gamma"], class_weight=best_svm_params["class_weight"], probability=True, random_state=21)


svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)
y_pred_prob = svm_model.predict_proba(X_test)

accuracy = accuracy_score(y_test, y_pred)

precision = precision_score(y_test, y_pred, average="weighted", zero_division=0)
recall = recall_score(y_test, y_pred, average="weighted")

roc_auc = roc_auc_score(y_test, y_pred_prob, average="weighted", multi_class="ovr")


In [14]:
print(f"accuracy is {accuracy:.5f}")
print(f"precision is {precision:.5f}")
print(f"recall is {recall:.5f}")
print(f"roc_auc is {roc_auc:.5f}")

accuracy is 0.88757
precision is 0.89267
recall is 0.88757
roc_auc is 0.98168


## 3. Decision tree

1. The same task for decision tree

In [17]:
dt_model = DecisionTreeClassifier(
    class_weight='balanced',
    criterion='gini',
    max_depth=22,
    random_state=21
)

dt_model.fit(X_train, y_train)

y_pred = dt_model.predict(X_test)
y_pred_prob = dt_model.predict_proba(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted", zero_division=0)
recall = recall_score(y_test, y_pred, average="weighted")
roc_auc = roc_auc_score(y_test, y_pred_prob, average="weighted", multi_class="ovr")

print(f"accuracy is {accuracy:.5f}")
print(f"precision is {precision:.5f}")
print(f"recall is {recall:.5f}")
print(f"roc_auc is {roc_auc:.5f}")

accuracy is 0.89053
precision is 0.89262
recall is 0.89053
roc_auc is 0.93806


## 4. Random forest

1. The same task for random forest.

In [20]:
rf_model = RandomForestClassifier(
    class_weight=None,
    criterion='gini',
    max_depth=28,
    n_estimators=50,
    random_state=21
)

rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)
y_pred_prob = rf_model.predict_proba(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average="weighted", zero_division=0)
recall = recall_score(y_test, y_pred, average="weighted")
roc_auc = roc_auc_score(y_test, y_pred_prob, average="weighted", multi_class="ovr")

print(f"accuracy is {accuracy:.5f}")
print(f"precision is {precision:.5f}")
print(f"recall is {recall:.5f}")
print(f"roc_auc is {roc_auc:.5f}")

accuracy is 0.92899
precision is 0.93009
recall is 0.92899
roc_auc is 0.99151


## 5. Predictions

1. Choose the best model.
2. Analyze: for which `weekday` your model makes the most errors (in % of the total number of samples of that class in your full dataset), for which `labname` and for which `users`.
3. Save the model.

In [22]:
errors = y_test != y_pred

test_data = X_test.copy()
test_data["true_label"] = y_test
test_data["predicted_label"] = y_pred
test_data["error"] = errors.astype(int)

In [30]:

error_analysis = pd.DataFrame({"true_weekday": y_test, "error": errors.astype(int)})
weekday_errors = error_analysis.groupby("true_weekday")["error"].sum()
weekday_total = error_analysis.groupby("true_weekday")["error"].count()
weekday_error_percent = (weekday_errors / weekday_total * 100).sort_values(ascending=False)
    
    
    
    
user_columns = [col for col in test_data.columns if "uid_user_" in col]
test_data["user_id"] = test_data[user_columns].idxmax(axis=1)  

user_errors = test_data.groupby("user_id")["error"].sum()
user_total = test_data.groupby("user_id")["error"].count()
user_error_percent = (user_errors / user_total * 100).sort_values(ascending=False)


lab_columns = [col for col in test_data.columns if "labname_" in col]
test_data["labname"] = test_data[lab_columns].idxmax(axis=1)

lab_errors = test_data.groupby("labname")["error"].sum()
lab_total = test_data.groupby("labname")["error"].count()
lab_error_percent = (lab_errors / lab_total * 100).sort_values(ascending=False)


    
weekday_error_percent.head(1), lab_error_percent.head(1), user_error_percent.head(1)

(true_weekday
 0    25.925926
 Name: error, dtype: float64,
 labname
 labname_lab03    100.0
 Name: error, dtype: float64,
 user_id
 uid_user_22    100.0
 Name: error, dtype: float64)

In [31]:
import joblib


joblib.dump(rf_model, "best_model.pkl")



['best_model.pkl']

## 6. Function

1. Write a function that takes a list of different models and a corresponding list of parameters (dicts) and returns a dict that contains all the 4 metrics for each model.

In [ ]:
def evaluate_models(models, params, X_train, y_train, X_test, y_test):    
    results = {}
    for model, param in zip(models, params):
        model_inst = model(**param)
        model_inst.fit(X_train, y_train)
        y_pred = model_inst.predict(X_test)
        y_prob = model_inst.predict_proba(X_test)
        
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred, average='weighted'),
            'recall': recall_score(y_test, y_pred, average='weighted'),
            'roc_auc': roc_auc_score(pd.get_dummies(y_test), y_prob, multi_class='ovr', average='weighted')
        }
        
        results[model.__name__] = metrics
        
    return results

In [ ]:
# example callevaluate_models(
    [SVC],
    [{'kernel':'rbf', 'C':10, 'gamma':'auto', 'class_weight':None, 'probability':True, 'random_state':21}],
    X_train, y_train, X_test, y_test
)